In [2]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
import os
import ee
import geemap
import ipywidgets as widgets
import pandas as pd
import numpy as np

In [4]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=fbMcanliK7tofaf_pLbhNo_-hJ_RpfeF4tmjwwtz71k&tc=IvGrpQjcbtKjsI8jm9ObcVj5Cvim_I_1A7fftM3IUoY&cc=LWplo9GJMLKLD2kgEFNciGYP1w2uUa6EHqW3gTShMvY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh47XcTcxSNcqm3vTweBBIrYcJpV5a9oZq6p4FVvffEIHavB4rjvZkU

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
!pip install geehydro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10140 sha256=f6c2c5cb2c507f985dc557cb3bf3271161585b666cb4485c11c695ffd3e98827
  Stored in directory: /root/.cache/pip/wheels/ff/aa/13/b6c5b687208b545f735833f12891f3154c76fb63f2cdd9f192
Successfully built geehydro


In [7]:
import ee, datetime
import pandas as pd
import numpy as np
import folium
import geehydro
import datetime as dt
import IPython.display

In [8]:
# start_date = '2021-01-01'
# end_date = '2021-06-30'

hussiantalab = folium.Map(location=[23.264998, 77.395892], zoom_start=10)

In [9]:
hussiantalab

In [10]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate("2015-01-01","2023-01-01")
hussain_AOI = ee.Geometry.Polygon([[77.395908, 23.264982],
          [77.395897, 23.265948],
          [77.396026, 23.265870],
          [77.396348, 23.265712],
          [77.396396, 23.265209],
          [77.396427, 23.265023],
          [77.396208, 23.264367],
          [77.396121, 23.263967],
          [77.395955, 23.263837],
          [77.395805, 23.263750],
          [77.395739, 23.264085],
          [77.395693,23.264406],
          [77.395616, 23.264894 ]])

sentinel_AOI = sentinel.filterBounds(hussain_AOI)

In [11]:
print('Total number of images :', sentinel_AOI.size().getInfo())

Total number of images : 166


Landsat

In [12]:
def addNDCI(image):
  ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
  return image.addBands(ndci)
with_ndci = sentinel_AOI.map(addNDCI)  

In [13]:
def meanNDCI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDCI').getInfo()

In [14]:
Images_ndci = with_ndci.select('NDCI').toList(with_ndci.size())
ndci_coll = []
for i in range(Images_ndci.length().getInfo()):
  image = ee.Image(Images_ndci.get(i-1))
  temp_ndci = meanNDCI(image)
  ndci_coll.append(temp_ndci)

## Getting Dates from the Image Collection

In [15]:
dates = np.array(with_ndci.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Creating Dataframe

In [16]:
df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
df.sort_index(ascending = True, inplace = True)

In [17]:
df

,Chlorophyll
2018-12-18,0.059086
2018-12-23,0.120644
2018-12-28,0.185665
2019-01-02,0.165246
2019-01-07,0.192004
...,...
2022-12-02,0.037268
2022-12-07,0.048930
2022-12-17,0.049954
2022-12-22,0.052176


In [18]:
df.to_csv('Chlorophyll.csv')

## NDTI Calculation

In [19]:
def addNDTI(image):
  ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
  return image.addBands(ndti)
with_ndti = sentinel_AOI.map(addNDTI)

In [20]:
def meanNDTI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDTI').getInfo()

In [21]:
Images_ndti = with_ndti.select('NDTI').toList(with_ndti.size())
ndti_coll = []
for i in range(Images_ndti.length().getInfo()):
  image = ee.Image(Images_ndti.get(i-1))
  temp_ndti = meanNDTI(image)
  ndti_coll.append(temp_ndti)

In [22]:
dates = np.array(with_ndti.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [23]:
df1 = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
df1.index = pd.to_datetime(df1.index, format="%Y/%m/%d")
df1.sort_index(ascending = True, inplace = True)

In [24]:
df1

,Turbidity
2018-12-18,-0.017512
2018-12-23,-0.099813
2018-12-28,-0.128578
2019-01-02,-0.081433
2019-01-07,-0.098589
...,...
2022-12-02,-0.030352
2022-12-07,-0.021043
2022-12-17,-0.025999
2022-12-22,-0.033139


In [25]:
df1.to_csv('Turbidity.csv')

In [26]:
def addNDSI(image):
  ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
  return image.addBands(ndsi)
with_ndsi = sentinel_AOI.map(addNDSI)

In [27]:
def meanNDSI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDSI').getInfo()

In [28]:
Images_ndsi = with_ndsi.select('NDSI').toList(with_ndsi.size())
ndsi_coll = []
for i in range(Images_ndsi.length().getInfo()):
  image = ee.Image(Images_ndsi.get(i-1))
  temp_ndsi = meanNDSI(image)
  ndsi_coll.append(temp_ndsi)

In [29]:
dates = np.array(with_ndsi.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [30]:
df2 = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinty'])
df2.index = pd.to_datetime(df2.index, format="%Y/%m/%d")
df2.sort_index(ascending = True, inplace = True)

In [31]:
df2

,Salinty
2018-12-18,0.043552
2018-12-23,0.126919
2018-12-28,0.029951
2019-01-02,0.087383
2019-01-07,0.043321
...,...
2022-12-02,0.025220
2022-12-07,0.019142
2022-12-17,0.026828
2022-12-22,0.024851


In [32]:
df2.to_csv('Salinty.csv')

## Dissolved Oxygen Calculation

In [33]:
def addDO(image):
  do = image.normalizedDifference(['B4', 'B5']).rename('DO')
  return image.addBands(do)
with_do = sentinel_AOI.map(addDO)

In [34]:
def meanDO(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('DO').getInfo()

In [35]:
Images_do = with_do.select('DO').toList(with_do.size())
do_coll = []
for i in range(Images_do.length().getInfo()):
  image = ee.Image(Images_do.get(i-1))
  temp_do = meanDO(image)
  do_coll.append(temp_do)

In [36]:
dates = np.array(with_do.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [37]:
df4 = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
df4.index = pd.to_datetime(df4.index, format="%Y/%m/%d")
df4.sort_index(ascending = True, inplace = True)

In [38]:
df4

,Dissolved Oxygen
2018-12-18,-0.062720
2018-12-23,-0.129145
2018-12-28,-0.205247
2019-01-02,-0.177468
2019-01-07,-0.206343
...,...
2022-12-02,-0.042336
2022-12-07,-0.053930
2022-12-17,-0.054804
2022-12-22,-0.056818


In [39]:
df4.to_csv('Dissolved Oxygen.csv')

## pH

In [40]:
def addpH(image):
  ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
  return image.addBands(ph)
with_pH = sentinel_AOI.map(addpH)

In [41]:
def meanpH(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('PH').getInfo()

In [42]:
Images_ph = with_pH.select('PH').toList(with_pH.size())
ph_coll= []
for i in range(Images_ph.length().getInfo()):
  image = ee.Image(Images_ph.get(i-1))
  temp_ph = meanpH(image)
  ph_coll.append(temp_ph)

In [43]:
dates = np.array(with_pH.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [44]:
df6 = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [45]:
df6

,pH
2018-12-18,7.622971
2018-12-23,7.651878
2018-12-28,7.667203
2019-01-02,7.757467
2019-01-07,7.741976
...,...
2022-12-02,7.546838
2022-12-07,7.608214
2022-12-17,7.631295
2022-12-22,7.561551


In [46]:
df6.to_csv('pH.csv')

## Temperature

In [47]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").\
              filterDate("2016-01-01","2022-01-01")
hussain_AOI = ee.Geometry.Polygon([[77.395908, 23.264982],
          [77.395897, 23.265948],
          [77.396026, 23.265870],
          [77.396348, 23.265712],
          [77.396396, 23.265209],
          [77.396427, 23.265023],
          [77.396208, 23.264367],
          [77.396121, 23.263967],
          [77.395955, 23.263837],
          [77.395805, 23.263750],
          [77.395739, 23.264085],
          [77.395693,23.264406],
          [77.395616, 23.264894 ]])

landsat_AOI = landsat.filterBounds(hussain_AOI)

In [48]:
print('Total number of images :', landsat_AOI.size().getInfo())

Total number of images : 230


In [49]:
def addtemp(image):
  temp = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temperature')
  return image.addBands(temp)
with_temp = landsat_AOI.map(addtemp)


In [50]:
def meantemp(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['temperature']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('temperature').getInfo()

In [51]:
Images_temp = with_temp.select('temperature').toList(with_temp.size())
temp_coll = []
for i in range(Images_temp.length().getInfo()):
  image = ee.Image(Images_temp.get(i-1))
  tempe_temp = meantemp(image)
  temp_coll.append(tempe_temp)

In [52]:
dates = np.array(with_temp.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [53]:
df5 = pd.DataFrame(temp_coll, index = day, columns = ['Temperature'])
df5.index = pd.to_datetime(df5.index, format="%Y/%m/%d")
df5.sort_index(ascending = True, inplace = True)

In [54]:
df5

,Temperature
2016-01-07,18.415886
2016-01-14,23.310221
2016-01-23,23.798392
2016-01-30,24.023891
2016-02-08,21.795062
...,...
2021-11-27,25.408819
2021-12-06,28.875814
2021-12-13,27.751886
2021-12-22,26.766852


In [55]:
df5.to_csv('Temperature.csv')

## DOM

In [56]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2018-08-01","2018-12-31")

hussain_AOI = ee.Geometry.Polygon([[77.395908, 23.264982],
          [77.395897, 23.265948],
          [77.396026, 23.265870],
          [77.396348, 23.265712],
          [77.396396, 23.265209],
          [77.396427, 23.265023],
          [77.396208, 23.264367],
          [77.396121, 23.263967],
          [77.395955, 23.263837],
          [77.395805, 23.263750],
          [77.395739, 23.264085],
          [77.395693,23.264406],
          [77.395616, 23.264894 ]])

sentinel3_AOI = sentinel3.filterBounds(hussain_AOI)

In [57]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

Total number of images : 492


In [58]:
def addDM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
  return image.addBands(dm)
with_dm = sentinel3_AOI.map(addDM)

In [59]:
def meanDM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('dom').getInfo()

In [60]:
Images_dm = with_dm.select('dom').toList(with_dm.size())
dm_coll= []
for i in range(Images_dm.length().getInfo()):
  image = ee.Image(Images_dm.get(i-1))
  temp_dm = meanDM(image)
  dm_coll.append(temp_dm)

In [67]:
df9 = pd.DataFrame(dm_coll, index = day, columns = ['Dissolved Oxygen Matter'])
df9.index = pd.to_datetime(df9.index, format="%Y/%m/%d")
df9.sort_index(ascending = True, inplace = True)

ValueError: ignored

In [ ]:
df6

In [ ]:
df6.to_csv("dom\AUG1_2018toDEC31_2018")

## SM

In [62]:
def addSM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  return image.addBands(suspended_matter)
with_sm = sentinel3_AOI.map(addSM)

In [63]:
def meanSM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = hussain_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('suspended_matter').getInfo()

In [68]:
Images_sm = with_sm.select('suspended_matter').toList(with_sm.size())
sm_coll= []
for i in range(Images_sm.length().getInfo()):
  image = ee.Image(Images_sm.get(i-1))
  temp_sm = meanSM(image)
  sm_coll.append(temp_sm)

In [69]:
df7 = pd.DataFrame(dm_coll, index = day, columns = ['Suspended Matter'])
df7.index = pd.to_datetime(df7.index, format="%Y/%m/%d")
df7.sort_index(ascending = True, inplace = True)

ValueError: ignored

In [ ]:
df7

In [ ]:
df7.to_csv("suspendedmatter.csv")